In [0]:
# Databricks notebook source
# Cell 1: Securely Configure Storage Access
#scope_name = "trubrixai-lab-scope"
#secret_name = "trubrixaidevst-access-key"
#storage_account_name = "trubrixaidevst"

#storage_account_key = dbutils.secrets.get(scope=scope_name, key=secret_name)

#spark.conf.set(
#  f"fs.azure.account.key.{storage_account_name}.dfs.core.windows.net",
#  storage_account_key
#)
#print("Configuration set securely for this notebook session.")

# COMMAND ----------



In [0]:
# Cell 2: Define Paths for Staging and Bronze Landing Zone
# The source directory in DBFS where we upload files manually
source_directory = "/FileStore/tables/"

# A single, consistent destination folder for our daily batch.
destination_directory = "abfss://bronze@trubrixaidevst.dfs.core.windows.net/shipments/daily_batch/"

print(f"Source Staging Area: {source_directory}")
print(f"Bronze Landing Zone: {destination_directory}")

# COMMAND ----------



In [0]:
# Cell 3: Clean the Bronze Landing Zone (CRITICAL STEP)
# To ensure we only process the new batch, we first delete all contents
# from our landing zone. This guarantees a clean slate for every run.
print(f"Cleaning previous batch from: {destination_directory}")
try:
  dbutils.fs.rm(destination_directory, recurse=True)
  print("Landing zone cleaned successfully.")
except Exception as e:
  print(f"Landing zone was already empty or did not exist, which is okay: {e}")

# Re-create the empty directory
dbutils.fs.mkdirs(destination_directory)
print("Landing zone is ready for new batch.")

# COMMAND ----------



In [0]:
# Cell 4: Move the New Batch of Files
# Now we move the new files from the staging area into our clean landing zone.
print("\nMoving new batch of files...")
for file in dbutils.fs.ls(source_directory):
  print(f"Moving file: {file.name}")
  dbutils.fs.mv(file.path, destination_directory + file.name)

print(f"Successfully moved all files to the landing zone.")

# COMMAND ----------

# Cell 5: Verify the Files Arrived
print("\nVerifying files in the landing zone:")
display(dbutils.fs.ls(destination_directory))